# Computer Lab: A case study in fitting spectra and identifying peaks
---

In this notebook, we present a sequence of exercises centered around the task of fitting spectra and identifying peaks in spectra. This problem is ubiquitous throughout all of science and many different approaches have been developed and can be used to tackle it. These approaches differ in what type of information is provided as prior knowledge or hyper-parameters provided to fitting model. For example, many methods center around an informed scientist either knowing roughly where peaks are located in the spectra -- either by looking at the spectra or through prior knowledge. Other methods attempt to find the peak locations and other parameters to the model and are only given a rough window of where such peaks can occur. In this notebook, we'll explore a few of these models, and use this example to recap many of the topics we've encountered in this class.

# Loading the data

The file `data/raman1.csv` contains Raman spectrum intensities $y$ for several Raman shift wavenumbers $x$ (which are labeled as "Pixels" in the csv file). Load this data into a `numpy` array so that each row corresponds to a wavenumber/intensity pair $(x, y)$.

For now, we're interested in a specific region of the Raman spectrum between wavenumbers $x = 450$ cm$^{-1}$ and 600 cm$^{-1}$. This is the region containing Silicon peaks, and knowing where these peaks are can be used to identify substrate temperatures. Extract the subset of rows of your data matrix corresponding to this region. Call this data `raw_data`

Plot the spectrum in this region. That is, plot the intensities vs. wavenumbers.

# Subtract background signal

The Raman spectrum has a background signal added to it, which originates from the use of specific equipment and other factors. We can view this background as a low-degree polynomial that is a function of wavenumber. This polynomial should be fitted against those data points of relatively low intensity values (i.e. everything but the actual peaks of the data).

Extract the data points with intensity values in the lower 25 percentile. Call these extracted rows `background_data`

Fit a quadratic polynomial to this extracted data

Obtain the background signal. Evaluate the polynomial for every wavenumber in the `raw_data` set. This will result in a background intensities. that can be subtracted from the intensities in the `raw_data` dataset.

Subtract the background signal from the intensities in `raw_data`. Call the resulting background subtracted data simply `data`.

# Define basis functions for peaks

We'll think of the background-subtracted spectrum in `data` as a linear combination of various Gaussian peaks 

$$ f(\omega) = \sum_{i=1}^k \theta_i g(\omega; \mu_i, \sigma_i).$$

where $\theta_i$ are coefficients and 

$$ g(\omega; \mu, \sigma) = \exp\left[-\frac{(\omega - \mu)^2}{\sigma^2}\right]$$

This is similar to a Gaussian mixture model, but here we're not viewing the mixtures of Gaussian peaks as a probability density function and so we don't require the area under such a mixture to be equal to 1. For the same reason, we don't include the typical factor out in front of the Gaussian peak function. This is also sometimes called a (Gaussian) radial basis function (RBF) model.

Define a Python function `g(omega, mu, sigma)` that implements the formula above for the Gaussian peak `g(\omega; \mu \sigma)`.

In [ ]:
def g(omega, mu, sigma):
    # your code here

# RBF regression

In the above model for the spectrum, $k$ is the number of peaks, while $\mu_i$ and $\sigma_i$ are the center and width of the $i$-th peak, respectively. The $\mu_i$ and $\sigma_i$ have units cm$^{-1}$ for Raman spectra.

First, we'll asssume that we know for this example that $k=2$, and we know the peak centers $\mu_1, \mu_2$ and widths $\sigma_1, \sigma_2$ -- you'll have to estimate these manually from the plot of the spectra. Thus, we can define two basis functions 

$$ \varphi_i(\omega) = g(\omega; \mu_i, \sigma_i) \quad \text{for } i = 1, 2.$$

Form vectors `phi_1` and `phi_2` corresponding to the two basis functions evaluated at each of the wavenumbers in `data`. (You should be using your function `g` defined above, along with the specific values of $\mu_i$ and $\sigma_i$ you estimated from the plots)

Using these vectors, form the design matrix $\Phi$, and then perform a linear regression to identify the least-squares estimator $\hat{\boldsymbol \theta}_\text{LS} = (\theta_1, \theta_2)^T$ for the solution to 

$$ \boldsymbol y = \Phi \boldsymbol \theta.$$

where $\boldsymbol y$ is the vector of intensities in the `data` dataset.

Look at the values you obtain for the $\theta_i$. What do they represent? Does it make sense given the specific spectrum we're fitting?

Obtain the model predictions of intensities uing the least-squares estimator.

Plot the model predictions along with the data in `data`. Use a solid line to plot the model predictions and a scatter plot to plot the data.

# LASSO-based peak identification

The above method assumes we know the peak centers and widths, and required a human to provide these to a model. Thus, the above method is useful in the scenario where you know the peaks of your spectra, but you are trying to identify the exact contribution of each peak toward the observed spectral intensities.

We now consider the problem of where we do not assume to know which peaks are present in the spectrum. One approach for this problem is take RBF regression to an extreme -- if we don't know which peaks contribute to the signal, just include an **exhaustive** set of peak basis functions. That is, instead of include the specific peaks of the signal (with known locations and widths), we'll simply build a set of basis functions from a combination of several peak locations and widths.

First, define a list of peak 150 locations between 450 cm$^{-1}$ to 600 cm^${-1}$. Name this list `locations`.

Now, similarly define a set of peak widths, from 1 cm$^{-1}$ to 20 cm$^{-1}$. Name this list `widths`.

We now want to for a list of location/widths pairs $(\mu_j, \sigma_j)$. You can do this with the `product` function in the `itertools` module. 

In [ ]:
import itertools

peak_parameters = np.array([ [mu, sigma] for mu,sigma in itertools.product(locations, widths)])

Print the shape of `peak_parameters` to make sure you understand what the rows and columns are.

Thus the $j$-th row of peak_parameters correspond to the $j$-th pair of parameters $(\mu_j, \sigma_j)$. Print out $\mu_{10}$ and $\sigma_{10}$.

Evaluate the wavenumbers in `data` on the function $\varphi_{10}(\omega) = g(\omega; \mu_{10}, \sigma_{10})$

Plot corresponding this basis function.

Define a function `form_design_matrix(omegas, peak_parameters)` that returns the design matrix, where the $j$-th column of the design matrix is just $\varphi_j(\omega)$ evaluated at every wavenumber in `omegas`.

In [ ]:
def form_design_matrix(omegas, peak_parameters):
    # Your code goes here.

Use your function to define the design matrix evaluated at every wavenumber in `data`.

Once you have the design matrix,$\Phi$, solve $\boldsymbol y = \Phi \boldsymbol \theta$ using the LASSO method, for the corresponding intensity data values $\boldsymbol x$. To do this, use a LASSO hyperparameter value $\lambda$ of your choice. Save and plot the LASSO estimator $\hat{\boldsymbol \theta}_\text{LASSO}$.

Repeat the above, but for different values of the hyperparameter $\lambda$. Make sure the behavior makes sense to you.

# Resampling the data for better peak fits

One thing you may notice is that often, your fits fail to properly fit through the tops of the peaks. This is due to the fact that in your data has a lot more small values than large values. These large-values are the actual peaks, and what we truly care about, but the Loss function we use to train the model treats every data point with equal weight. This results in a decrease impact of the large-value data points in the fit.

We can artificially correct this in several ways, which we'll explore in-depth when we discuss classification (and the analogous concept of **class imbalance**). Here we'll explore one way to do this, using **data resampling**.

With resampling, we sample a training data set from the original data so that the sampled training data set has a better of between intensity values. To do this, we:

1. Bin (equivalently: group or partition) all the data points according to their intensity values. 
2. Sample (with replacement) a fixed number of points from each bin.

Thus, if we have 10 bins, and we sample 5 points per bin, we'll get a training data set of 50 points with a much better balance of intensity values.

First, we'll define our bin edges, $\boldsymbol b = (b_0, b_1, ..., b_n)$. Define a vector `bin_edges` to be 20 equally spaced points between the smallest intensity value in `data` and the largest intensity value.

Once we define the bin edges, we can then partition the data into $n$ bins. Specifically we'll assign to the $i$-th bin exactly the data points whose intensity values are between $b_i$ and $b_{i+1}$, for $i = 0, 1, ..., n-1$. Do do this quickly, we can use the `numpy` function `np.digitize` that takes in a set of values, `values`, and bin edges, and returns an index array of the same length as `values` where the $j$-th entry is the bin number for the $j$-th value in `value`.

Using the intensity values in `data`, the `bin_edges` array defined above and `np.digitize`, define the index array of bins for each data point, and call this array `bin_indices`.

Using `bin_indices`, extract the data points from `data`in the $i = 3$rd bin.

Using the [`choice` function](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html) from the `numpy` random number generator, sample 5 data points from this bin with replacement. You may need to read the documentation to make sure you are using the function correctly.

Once we have have `bin_indices` and the ability to extract the data points from `data` corresponding to any bin we wish, we can start forming our training data.

For now, we'll sample 5 data points from each of the 20 bins, with replacement. Write a loop to extract 5 data points for each of the bins, and save these data points to an `numpy` array named `training_data`.

Once the training data, repeat the procedure in the previous section to train a new LASSO fit based on this `training_data` dataset. Again, play around with the regularization parameter and note any interesting observations you may have.

# An information criterion for picking the regularization parameter
---

Another method to pick hyperparameters other than the normal validation-based method is to use an information criterion, which is a measure of the balance between model complexity and accuracy. Such criteria are often easy-to-calculate short-cuts to performing a full validation of your models. By achieving a good balance, you prevent overfitting by using relatively simple models (e.g. a reasonable number of basis functions), but you also achieve an relatively good fit to your data. Both the use of validation and information criteria help you perform **model selection**.

There are several examples of such information criteria, such as the Akaike Information Criterion (AIC) and the Bayesian Information Criterion (BIC). Here we'll calculate one where the number of relevant basis functions identified by the LASSO procedure serves as the measure for model complexity. 



Fit a LASSO model using the above procedure, and identify the fitted coefficient values that are significantly large, say anything past the median value of all coefficient values. We'll call these values $\theta_i$ and the corresponding basis functions $\varphi_i$ synonymously as the **significant** basis functions obtained by the LASSO procedure.

Count how many significant basis functions there are.

For the fitted LASSO model, compare the model predictions with the data in `data`. Use this comparison to calculate the sum-of-squared-error value for this model.

Repeat for different values of $\alpha$ on a logarithmic scale. For each selection of $\alpha$, calculate the number of significant basis functions (complexity) and sum-of-squared error value (accuracy). Store these numbers in separate arrays.

Normalize the two arrays so that the numbers range from between 0 and 1. This allows us to compare the two metrics of the models.

Plot the normalized values versus the values of $\alpha$ you considered. You may want to plot the horizontal axis on a logarithmic scale using `plt.scale('log')`.

Using your plot, identify the value of $\alpha$ where the normalized complexity equals the normalized accuracy. This value is the setting of the LASSO regularization parameter that balances between complexity and accuracy. What is this value?

Use this optimized value of $\alpha$ for the next section.

# Clustering to identify peaks
---

Plot the significant basis functions scaled by the corresponding coefficient. That is, plot $\theta_i \cdot g(\omega; \mu_i, \sigma_i)$.

You should note how the peaks cluster. Specifically, Use the location values $\mu_i$ for the significant peaks to do a 1D clustering (using $k=2$ clusters). These clusters roughly correspond to the peaks of the spectrum.

Which basis functions $\varphi_i = g(\omega; \mu_i, \sigma_i)$ correspond to peak locations in the first cluster? Print out the indices of such functions.

Combine all significant basis function for the first cluster. That is, form the sum $\sum_{i} \theta_i g(\omega; \mu_i, \theta_i)$ for every significant basis function $\varphi_i$ in the first cluster, for every intensity value in `data`. Plot the sum.

Repeat for the rest of the peaks to get a plot of all the peaks together.

Calculate the silhouette score for your clustering.

Perform another clustering on the peak locations $\mu_i$ but try $k=3$. Calculate the silhoutte score for this.

Repeat this again, but this time for $k = 4, 5, 6$. Calculate the silhoutte score for each choice of $k$. Which one is the largest?

Use the $k$ with the largest value as the optimized value of the number of clusters, which is analogous to the number of actual peaks in the spectrum. Does this match what you can see with your own eyes?

Plot the fitted spectrum with the optimized value of $k$ and $\alpha$, and plot each peak, along with their corresponding component basis functions.